# Numeric optimization

To find the optimal weights of the logistic regression, we can use {prf:ref}`gradient descent <GD>` algorithm. To apply this algorithm, one need to calculate the gradient of the loss function.

## Binary logistic regression

Multiply the loss function {eq}`bin-log-reg-loss` by $n$:

$$
\mathcal L(\boldsymbol w) = 
-\sum\limits_{i=1}^n \big(y_i \log(\sigma(\boldsymbol x_i^\top \boldsymbol w)) + (1- y_i)\log(1 - \sigma(\boldsymbol x_i^\top \boldsymbol w))\big).
$$

To find $\nabla \mathcal L(\boldsymbol w)$ observe that

$$
   \nabla \log(\sigma(\boldsymbol x_i^\top \boldsymbol w)) = \frac {\nabla \sigma(\boldsymbol x_i^\top \boldsymbol w)}{\sigma(\boldsymbol x_i^\top \boldsymbol w)} = 
   \frac{\sigma'(\boldsymbol x_i^\top \boldsymbol w) \nabla(\boldsymbol x_i^\top \boldsymbol w)}{{\sigma(\boldsymbol x_i^\top \boldsymbol w)}}.
$$

Also,

$$
   \nabla \log(1 - \sigma(\boldsymbol x_i^\top \boldsymbol w)) = -\frac {\nabla  \sigma(\boldsymbol x_i^\top \boldsymbol w)}{1 - \sigma(\boldsymbol x_i^\top \boldsymbol w)} = 
   \frac{\sigma'(\boldsymbol x_i^\top \boldsymbol w) \nabla(\boldsymbol x_i^\top \boldsymbol w)}{{1 - \sigma(\boldsymbol x_i^\top \boldsymbol w)}}.
$$

**Q**. What is $\nabla(\boldsymbol x_i^\top \boldsymbol w)$?

Putting it altogeter, we get

$$
   \nabla \mathcal L(\boldsymbol w) = -\sum\limits_{i=1}^n \big(y_i(1 - \sigma(\boldsymbol x_i^\top \boldsymbol w))\boldsymbol x_i - (1-y_i)\sigma(\boldsymbol x_i^\top \boldsymbol w)\boldsymbol x_i\big) = \sum\limits_{i=1}^n (\sigma(\boldsymbol x_i^\top \boldsymbol w) - y_i)\boldsymbol x_i.
$$

````{admonition} Question
:class: important
How to write $\nabla \mathcal L(\boldsymbol w)$ as a product of a matrix and a vector, avoiding the explicit summation?

```{hint}
:class: dropdown
The shape of $\nabla \mathcal L(\boldsymbol w)$ is the same as of $\boldsymbol w$, i.e., $d\times 1$. Now observe that

$$
   \begin{pmatrix}
   \sigma(\boldsymbol x_1^\top \boldsymbol w) - y_1 \\
   \vdots \\
   \sigma(\boldsymbol x_n^\top \boldsymbol w) - y_n
   \end{pmatrix}
   = \sigma(\boldsymbol X \boldsymbol w )- \boldsymbol y \in \mathbb R^n.
$$

What should we multiply by this vector to obtain $\nabla \mathcal L$?
```
````

````{admonition} Question
:class: important
 What is hessian $\nabla^2 L(\boldsymbol w)$?

```{admonition} Answer
:class: tip, dropdown
$$
\nabla^2 L(\boldsymbol w) = \boldsymbol X^\top \boldsymbol S \boldsymbol X,
$$

where

$$
   \boldsymbol S = \mathrm{diag}\{\sigma(\boldsymbol X \boldsymbol w )- \boldsymbol y\} = \begin{pmatrix}
   \sigma(\boldsymbol x_1^{\boldsymbol{\top}} \boldsymbol w) - y_1  & \ldots & 0 \\
   \vdots & \ddots & \vdots \\
   0 & \ldots & \sigma(\boldsymbol x_n^{\boldsymbol{\top}} \boldsymbol w) - y_n
   \end{pmatrix}
$$
```
````

## Breast cancer dataset: numeric optimization 

Fetch the dataset:

In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target
X.shape, y.shape

((569, 30), (569,))

Apply the {prf:ref}`gradient descent <GD>` algorithm to the logistic regression:

In [2]:
import numpy as np
from scipy.special import expit

def logistic_regression_gd(X, y, C=1, learning_rate=0.01, tol=1e-3, max_iter=10000):
    w = np.random.normal(size=X.shape[1])
    gradient = X.T.dot(expit(X.dot(w)) - y) + C * w
    for i in range(max_iter):
        if np.linalg.norm(gradient) <= tol:
            return w
        w -= learning_rate * gradient
        gradient = X.T.dot(X.dot(w) - y) + C * w
    print("max_iter exceeded")
    return w

Fit the logistic regresion on the whole dataset:

In [3]:
w = logistic_regression_gd(X, y, learning_rate=1e-9, max_iter=10**5)
w

max_iter exceeded


array([ 0.95916668, -0.54046488, -0.49365735,  0.01951272,  1.16513025,
       -0.92025639, -0.35628036,  0.07792495, -0.08304842, -0.5860345 ,
        2.35522508, -0.6339002 ,  0.96174227, -0.04107821, -1.56653769,
       -0.12061115, -0.4241051 ,  0.29403223, -0.54488831, -0.87231864,
        0.20096651,  0.31404291,  0.31351565, -0.01779842, -1.00909062,
       -1.71039148, -0.00979913,  1.43453601, -0.38925477,  0.95908276])

Calculate the accuracy score:

In [4]:
from sklearn.metrics import accuracy_score
accuracy_score(expit(X.dot(w)) > 0.5, y)

0.5026362038664324

Compare with `sklearn`:

In [5]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(fit_intercept=False, max_iter=5000)
log_reg.fit(X, y)
print(log_reg.score(X, y))
print(accuracy_score(log_reg.predict(X), y))
log_reg.coef_

0.9595782073813708
0.9595782073813708


array([[ 2.1975179 ,  0.11135549, -0.07305484, -0.00335447, -0.16745265,
        -0.41425964, -0.67991182, -0.36939821, -0.2430434 , -0.02410403,
        -0.02336256,  1.20717466,  0.04631705, -0.09649416, -0.01903567,
         0.01810195, -0.03747183, -0.04324005, -0.04367351,  0.00878879,
         1.29284173, -0.33773315, -0.12388807, -0.02474166, -0.31037867,
        -1.14588466, -1.63908301, -0.70741377, -0.73405013, -0.11303725]])

In [6]:
np.linalg.norm(w - log_reg.coef_)

5.4344292111849075

## Multinomial logistic regression

Recall that the loss function in this case is

$$
    \begin{multline*}
    \mathcal L(\boldsymbol W) = -\sum\limits_{i=1}^n \sum\limits_{k=1}^K  y_{ik} \bigg(\boldsymbol x_i^\top\boldsymbol w_{k} -\log\Big(\sum\limits_{k=1}^K \exp(\boldsymbol x_i^\top\boldsymbol w_{k})\Big)\bigg) = \\
    =
    -\sum\limits_{i=1}^n \sum\limits_{k=1}^K  y_{ik} \bigg(\sum\limits_{j=1}^d x_{ij} w_{jk} -\log\Big(\sum\limits_{k=1}^K \exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)\Big)\bigg)
    \end{multline*}
$$

One can show that 

$$
    \nabla \mathcal L(\boldsymbol W) = \boldsymbol X^\top (\boldsymbol {\widehat Y} - \boldsymbol Y) = \boldsymbol X^\top ( \sigma(\boldsymbol{XW}) - \boldsymbol Y).
$$

<!-- Observe that

$$
    \frac{\partial}{\partial w_{pq}} (x_{ij} w_{jk}) = x_{ij} \delta_{pj} \delta_{qk},
$$

$$
\frac{\partial}{\partial w_{pq}}\bigg(\log\Big(\sum\limits_{k=1}^K \exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)\Big)\bigg) = \frac{\exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)}{\sum\limits_{k=1}^K \exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)} x_{ip} \delta_{qk}
$$

Hence, 

$$
    \frac{\partial \mathcal L}{\partial w_{pq}} = \sum\limits_{i=1}^n \sum\limits_{k=1}^K y_{ik}\bigg(\sum\limits_{j=1}^d  \bigg(  x_{ij} \delta_{pj} \delta_{qk} - \frac{\exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)}{\sum\limits_{k=1}^K \exp\Big(\sum\limits_{j=1}^d x_{ij} w_{jk}\Big)} x_{ip} \delta_{qk}\bigg)\bigg)
$$ -->

```{admonition} TODO
:class: warning
* Try numerical optimization on several datasets
* Apply Newton's method and compare it's performance with GD
```